In [ ]:
#importação da biblioteca pandas
import pandas as pd 

In [ ]:
#leitura dos arquivos
  #detalhamento.xlsx
df_am = pd.read_excel('detalhamento.xlsx', sheet_name = 'AM', skiprows = 1)
df_rr = pd.read_excel('detalhamento.xlsx', sheet_name= 'RR', skiprows= 1)
df_ro = pd.read_excel('detalhamento.xlsx', sheet_name= 'RO', skiprows= 1)
df_ac = pd.read_excel('detalhamento.xlsx', sheet_name= 'AC', skiprows= 1)
  #vendas.csv
df_vendas = pd.read_csv('vendas.csv', sep= '|', skiprows= 0, encoding =  'utf-8')

In [ ]:
#tratamento de dados
  #renomear colunas
df_am.columns = ['loja', 'escritorio', 'uf', 'operadora', 'valor_bruto' ]
df_rr.columns = ['loja', 'escritorio', 'uf', 'operadora', 'valor_bruto' ]
df_ro.columns = ['loja', 'escritorio', 'uf', 'operadora', 'valor_bruto' ]
df_ac.columns = ['loja', 'escritorio', 'uf', 'operadora', 'valor_bruto' ]
df_vendas.columns = ['escritorio', 'operadora', 'material', 'data', 'valor_liquido' ]

In [ ]:
  #verificar tipos 
    # - escolhi somente uma para exemplificar e o código não ficar tão grande
  df_am.dtypes

In [ ]:
#alteração do campo 'valor_bruto' para float
df_am['valor_bruto'] = df_am['valor_bruto'].astype(float)
df_rr['valor_bruto'] = df_rr['valor_bruto'].astype(float)
df_ro['valor_bruto'] = df_ro['valor_bruto'].astype(float)
df_ac['valor_bruto'] = df_ac['valor_bruto'].astype(float)

In [ ]:
#deletar dados
df_am = df_am.drop(['uf'], axis = 'columns')
df_rr = df_rr.drop(['uf'], axis = 'columns')
df_ro = df_ro.drop(['uf'], axis = 'columns')
df_ac = df_ac.drop(['uf'], axis = 'columns')

In [ ]:
df_vendas = df_vendas.drop(['material', 'data'], axis = 'columns')

In [ ]:
#concatenar tabelas
df_geral = pd.concat([df_am, df_rr, df_ro, df_ac])

In [ ]:
df_geral = df_geral.reset_index(drop = True) #resetando o index

In [ ]:
#agrupamento de dados
df_vendas = df_vendas.groupby(['operadora', 'escritorio']).agg({'valor_liquido':'sum'}).reset_index()

In [ ]:
#correlação de tabelas
df_relatorio = pd.merge(df_geral, df_vendas, on = ['escritorio', 'operadora'], how = 'left')

In [ ]:
#regras de negócio
def status(v_bruto, v_liquido):
  if (v_bruto != v_liquido):
    return 'Dados divergentes'
  else: 
    return ' '

In [ ]:
df_relatorio['status'] = df_relatorio.apply(lambda row: status(row['valor_bruto'], row['valor_liquido']), axis = 'columns')

In [ ]:
df_relatoriofinal = df_relatorio[['loja', 'operadora', 'valor_liquido', 'valor_bruto', 'status']].copy()

In [ ]:
#exportar dados .XLSX
writer = pd.ExcelWriter('Relatorio_Final.xlsx')
df_relatoriofinal.to_excel(writer, sheet_name = 'relatorio', index = False)
writer.save()